In [ ]:
# Import the necessary packages and modules
# sci.mplstyle is customized Matplotlib style
import matplotlib.pyplot as plt
plt.style.use('../matplotlib/sci.mplstyle')
import numpy as np
from schrodinger import solve_schrodinger
from poisson import solve_poisson

# Set r from 0 to 15 bohr with 50000 steps
r, dr = np.linspace(0, 15, 50001, retstep=True)
r = r[1:]  # Skip r = 0

# Parameter for exchange potential Vx
C2 = -0.738;
# PZ parameters for correlation functional Vc 
Aa = 0.0311; B= -0.048; C= 0.002; D= -0.0116; gamma= -0.1423; beta1= 1.0529; beta2= 0.3334;

# The exchange potential
def exc(n):
    V_x = (4/3)*C2*n**(1/3)
    e_x = C2*n**(1/3)
    return (V_x, e_x)

# The correlation potential
def cor(n):
    rs = (3/4/np.pi/n)**(1/3)
    V_c = np.piecewise(rs, [rs<1,rs>=1],[lambda rs: Aa*np.log(rs)+B-Aa/3+2/3*C*rs*np.log(rs)+(2*D-C)*rs/3, lambda rs: gamma/(1+beta1*rs**(1/2)+beta2*rs)*(1+7/6*beta1*rs**(1/2)+beta2*rs)/(1+beta1*rs**(1/2)+beta2*rs)])
    e_c = np.piecewise(rs,[rs<1,rs>=1],[lambda rs: Aa*np.log(rs)+B+C*rs*np.log(rs)+D*rs, lambda rs: gamma/(1+beta1*rs**(1/2)+beta2*rs)])
    return (V_c, e_c)

def dft(V_en, maxiter=100, stop=0.001, correlation=False, verbose=False):
    V_H = np.zeros_like(V_en)
    V_x = np.zeros_like(V_en)
    V_c = np.zeros_like(V_en)
    e_x = np.zeros_like(V_en)
    e_c = np.zeros_like(V_en)
    eps = None
    
    for i in range(maxiter):
        eps_old = eps
        V = V_en + V_H + V_x + V_c
        V_xc = V_x + V_c
        e_xc = e_x + e_c
        
        # eigen energy and u(r)
        eps, u = solve_schrodinger(r, V)
        
        # normalize u(r)
        u /= np.linalg.norm(u)*np.sqrt(dr)
        
        # convergence of eigen energy
        if eps_old is not None:
            if verbose:
                print('Step %02d: eps = %f, |eps - eps_old| = %f' %(i, eps, abs(eps - eps_old)))
            if abs(eps - eps_old) < stop:
                return 2*eps - dr*np.dot(V_H, u**2) - 2*dr*np.dot(V_xc - e_xc, u**2)
        elif verbose:
            print
            
        # update Hartree potential
        U_H = solve_poisson(r, u)
        V_H = 2*U_H/r

        # total electron density
        n = 2*(u**2/4/np.pi/r**2)
        
        # update exchange potential
        V_x, e_x = exc(n)
        
        # update correlation potential
        if correlation:
            V_c, e_c = cor(n)
        elif correlation:
            V_c = 0
            e_c = 0

    raise Exception('Not converged after %d iterations; |eps - eps_old| = %f' %(maxiter, abs(eps - eps_old)))
    
print('Total energy of He without correlation')
print('JOB DONE: total energy E = %f Ha' %(dft(-2/r, correlation=False,verbose=True)))
print('--------------------------------------')
print('Total energy of He with correlation')
print('JOB DONE: total energy E = %f Ha' %(dft(-2/r, correlation=True, verbose=True)))